In [20]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config2 import g_key

In [21]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Esperance,AU,2021-10-09 20:43:27,-33.8667,121.9000,49.84,59,100,5.23
1,1,Nikolskoye,RU,2021-10-09 20:43:27,59.7035,30.7861,53.87,85,99,5.77
2,2,Arraial Do Cabo,BR,2021-10-09 20:38:57,-22.9661,-42.0278,69.76,68,90,16.11
3,3,Jumla,NP,2021-10-09 20:43:28,29.2747,82.1838,51.39,82,6,4.36
4,4,Wattegama,LK,2021-10-09 20:43:28,6.7989,81.4808,77.18,77,100,4.29


In [22]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [23]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat","Lng"]]

# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)

# 6. Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [24]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [26]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [27]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Wattegama,LK,2021-10-09 20:43:28,6.7989,81.4808,77.18,77,100,4.29
10,10,Guerrero Negro,MX,2021-10-09 20:43:29,27.9769,-114.0611,75.67,53,5,23.38
11,11,Russell,US,2021-10-09 20:43:30,32.3502,-85.1999,83.80,52,75,3.44
13,13,Alofi,NU,2021-10-09 20:43:05,-19.0595,-169.9187,78.69,83,75,5.75
15,15,Nago,JP,2021-10-09 20:43:31,26.5881,127.9761,82.47,80,6,18.19
24,24,Hilo,US,2021-10-09 20:43:24,19.7297,-155.0900,79.79,87,90,3.44
25,25,Kapaa,US,2021-10-09 20:43:33,22.0752,-159.3190,82.38,73,75,7.00
28,28,Tete,MZ,2021-10-09 20:43:34,-16.1564,33.5867,82.72,31,0,3.80
31,31,Eyl,SO,2021-10-09 20:43:34,7.9803,49.8164,77.45,83,10,12.95
32,32,Salalah,OM,2021-10-09 20:43:35,17.0151,54.0924,82.49,83,20,5.75


In [28]:
preferred_cities_df.count()

City_ID       172
City          172
Country       170
Date          172
Lat           172
Lng           172
Max Temp      172
Humidity      172
Cloudiness    172
Wind Speed    172
dtype: int64

In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Wattegama,LK,77.18,6.7989,81.4808,
10,Guerrero Negro,MX,75.67,27.9769,-114.0611,
11,Russell,US,83.80,32.3502,-85.1999,
13,Alofi,NU,78.69,-19.0595,-169.9187,
15,Nago,JP,82.47,26.5881,127.9761,
24,Hilo,US,79.79,19.7297,-155.0900,
25,Kapaa,US,82.38,22.0752,-159.3190,
28,Tete,MZ,82.72,-16.1564,33.5867,
31,Eyl,SO,77.45,7.9803,49.8164,
32,Salalah,OM,82.49,17.0151,54.0924,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [31]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping")


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [32]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Wattegama,LK,77.18,6.7989,81.4808,
10,Guerrero Negro,MX,75.67,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
11,Russell,US,83.80,32.3502,-85.1999,Macs
13,Alofi,NU,78.69,-19.0595,-169.9187,Taloa Heights
15,Nago,JP,82.47,26.5881,127.9761,Hotel Yugaf Inn Okinawa
24,Hilo,US,79.79,19.7297,-155.0900,Hilo Hawaiian Hotel
25,Kapaa,US,82.38,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
28,Tete,MZ,82.72,-16.1564,33.5867,Zambeze Paraíso Misterioso
31,Eyl,SO,77.45,7.9803,49.8164,Amiin Hotel
32,Salalah,OM,82.49,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"


In [33]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [35]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))